In [2]:
import numpy as np
import pandas as pd
import csv
from faker import Faker
import random

In [72]:
#ALL_DEFINED_FUNCTIONS_GOES_HERE
def dist(pos):
    [src_lat,src_lon,src_alt,dest_lat,dest_lon,dest_alt] = pos
    return np.sqrt(np.square(src_lat-dest_lat) + np.square(src_lon-dest_lon) + np.square(src_alt-dest_alt))
def calc_time(pos,speed):
    return int(dist(pos)/speed)
def is_close(pos):
    [src_lat,src_lon,src_alt,dest_lat,dest_lon,dest_alt] = pos
    return (dist(pos) < 0.1) #can be changed accordingly

def intrapol(pos,vel,acc,speed,maneuver_prob,maneuver_fact,noise_fact):
    global timestep, man_steps
    [src_lat,src_lon,src_alt,dest_lat,dest_lon,dest_alt] = pos
    [x_vel,y_vel,z_vel] = vel
    [x_acc,y_acc,z_acc] = acc
    lats , lons , alts = [src_lat] , [src_lon] , [src_alt]
    man = False
    i = 1
    while(not is_close([lats[-1],lons[-1],alts[-1],dest_lat,dest_lon,dest_alt])):
        print(lats[-1],lons[-1],alts[-1])
        if(man == True):
            x_vel = x_vel + (random.random() - 0.5)*2*maneuver_fact
            y_vel = y_vel + (random.random() - 0.5)*2*maneuver_fact
            z_vel = z_vel + (random.random() - 0.5)*2*maneuver_fact
            for k in range(man_steps):            
                lats.append(lats[-1] + x_vel*timestep)
                lons.append(lons[-1] + y_vel*timestep)
                alts.append(alts[-1] + z_vel*timestep)
            man = False
            i = i + man_steps
        else:
#             print(lats[-1],lons[-1],alts[-1])
            lats.append(lats[-1] + x_vel*timestep)
            lons.append(lons[-1] + y_vel*timestep)
            alts.append(alts[-1] + z_vel*timestep)
#             x_velp = x_vel
#             print(x_velp ,x_vel)
            pos_i = [lats[-1],lons[-1],alts[-1],dest_lat,dest_lon,dest_alt]
            x_vel = x_vel + (lats[-1]-dest_lat)/dist(pos) + (random.random()-0.5)* 2 * noise_fact/100000
            y_vel = y_vel + (lons[-1]-dest_lon)/calc_time(pos_i,speed) + (random.random()-0.5)* 2 * noise_fact/100000
            z_vel = z_vel + (alts[-1]-dest_alt)/calc_time(pos_i,speed) + (random.random()-0.5)* 2 * noise_fact/100000
            man = (random.random() <= maneuver_prob)
            i = i + 1
    lats.append(dest_lat)
    lons.append(dest_lon)
    alts.append(dest_alt)
    return {"pos":[lats,lons,alts],"vel":[x_vel,y_vel,z_vel]}

In [73]:
#all instantiation of classes goes here
fake = Faker()

In [74]:
#the rand factor is used for deviation in the path of the flight
rand_factor = 1/100       # 1/100 will include a maximum noice of 1000 Mts (basically rand_factor*1L)
print(random.random()*rand_factor)

0.0006927830001926127


In [75]:
print("Give the corresponding values")
print("<lat,lon,alt pairs> <totaltime in s> <total datapoints>")
# ipt = input()
# [m,t,n] = ipt.split(" ")
# m = int(m)
# t = int(t)
# n = int(n)
timestep = 1
man_steps = 5
m = 20
t = 3600
# n = 3600
speed = 1000 / 100000
avg_alt = 5000 / 100000
alt_var = 250 / 100000
take_off = False
maneuver = 0 #To be given a value between 0 and 1 like a boolean and intermediate values will define that much degree of maneuver
fighter = True
maneuver_fact = maneuver * 2 / 1000
maneuver_prob = 0 #set this to be the number of times your plane has to maneuver per 1 move setting this to zero will make no maneuver and 1 will make random motion towards the target dest
regular_time = True

Give the corresponding values
<lat,lon,alt pairs> <totaltime in s> <total datapoints>


In [76]:
#defining the source and destination in a random manner for now
#can be changed to some other method to make it some other small distance flyable objects
src_lat, src_lon , d_lat , d_lon = float(fake.latitude()), float(fake.longitude()), float(fake.latitude()), float(fake.longitude())
print(src_lat, src_lon , d_lat , d_lon)

-76.8091885 -2.851169 34.1158955 165.838667


In [77]:
#defining the path of latitude
def calc_pos(src_lat,src_lon,d_lat,d_lon):
    pts_lat=[src_lat]
    pts_lon=[src_lon]
    for i in range(m):
        new_src_lon = src_lon+(d_lon-src_lon+random.random()*rand_factor)/(m-i+1)
        pts_lon.append(new_src_lon)
        src_lon = new_src_lon
        new_src_lat = src_lat+(d_lat-src_lat+random.random()*rand_factor)/(m-i+1)
        pts_lat.append(new_src_lat)
        if(is_close([pts_lat[-1],pts_lon[-1],0,pts_lat[-2],pts_lon[-2],0])):
            return [[],[]]
        else:
            res = calc_pos(pts_lat[-1],pts_lon[-1],pts_lat[-2],pts_lon[-2])
            pts_lat.extend(res[0])
            pts_lon.extend(res[1])
        src_lat = new_src_lat
    pts_lat.append(d_lat)
    pts_lon.append(d_lon)
    return [pts_lat,pts_lon]

In [ ]:
print (calc_pos(src_lat, src_lon , d_lat , d_lon))

In [63]:
orig_lat = []
orig_lon = []
orig_alt = []
alt_src = avg_alt + (random.random()-0.5)*2*alt_var
if(take_off == False):
    vel = [random.random()*speed,None,0]
    vel[1] = np.sqrt(speed - np.square(vel[0]))
#     vel = [0,0,0]
#     print(vel)
    acc = [0,0,0]
    for i in range(len(pts_lat)-1):
        # print(pts_lat[i],pts_lat[i+1])
        alt_dst = avg_alt + (random.random()-0.5)*2*alt_var
        pos = [pts_lat[i],pts_lon[i],alt_src,pts_lat[i+1],pts_lon[i+1],alt_dst]
#         print(pos,vel,acc,speed,maneuver_prob,maneuver_fact)
        print("========================"+str((pts_lat[i],pts_lon[i]))+"=========================================")
        res = intrapol(pos,vel,acc,speed,maneuver_prob,maneuver_fact,0)
        vel = res["vel"]
        vect = res["pos"]
        orig_lat.extend(vect[0])
        orig_lon.extend(vect[1])
        orig_alt.extend(vect[2])
        
else:
    for i in range(len(pts_lat)-3):
        pass
# print(len(orig_lat))

========================(-6.851403604824257, -171.7434332120413)=========================================
========================(-6.6941681664600825, -171.95283258796468)=========================================
========================(-6.537333354646198, -172.1624165139158)=========================================
========================(-6.380303472776102, -172.37197612353881)=========================================
========================(-6.223443213172418, -172.58161755518466)=========================================
========================(-6.06635132535415, -172.7910285646842)=========================================
========================(-5.9095296996053746, -173.00027561078127)=========================================
========================(-5.752351475868636, -173.20960927471407)=========================================
========================(-5.594993274325752, -173.41916210979562)=========================================
========================(-5.43782103376

In [64]:
data = np.array([range(1,len(orig_alt)+1),orig_lat,orig_lon,orig_alt])
data = np.transpose(data)
print(data.shape)
data = np.multiply(data,[timestep,1,1,100000])
# print (data)

(42, 4)


In [27]:
d = pd.DataFrame(data=data,columns = ["time(in s)","lat","lon","alt"])
print (d)
d.to_csv('t11.csv', sep=",", index=False)
# with open('t10.csv', 'w', newline='') as file:
#     writer = csv.writer(file)
# #     writer.head()
#     writer.writerows(d) #, fieldnames = ["time","lat","lon","alt"]

     time(in s)        lat        lon          alt
0           1.0  56.203319  10.221528  4865.144611
1           2.0  56.206721  10.321470  4865.144611
2           3.0  56.210125  10.421412  4864.677128
3           4.0  56.213524  10.521357  4864.587426
4           5.0  56.216921  10.621307  4864.014797
..          ...        ...        ...          ...
142       143.0  40.605228  32.290117  4874.478022
143       144.0  40.608640  32.390033  4879.615250
144       145.0  40.612056  32.489949  4884.833896
145       146.0  40.615474  32.589863  4890.159974
146       147.0  39.815008  33.178559  5060.004774

[147 rows x 4 columns]
